In [ ]:
import time
import datetime
import pytz
import requests
import json
from base64 import b64encode
import os
from dotenv import load_dotenv
import sys

from google_cal_client import GoogleCalClient

In [ ]:
class DataNotTrustedException(Exception):
    """Custom exception to indicate that the data returned should not be trusted."""
    pass

class TogglClient:
    """For reading toggl time entries via the toggl API."""

    def __init__(self):
        self.TOGGL_API_URL = 'https://api.track.toggl.com/api/v9/me'

        # Load environment variables from the .env file
        load_dotenv()

        # Get the API key from the environment variables
        api_token_string = os.getenv('API_KEY')

        # Check if the API key was loaded correctly
        if api_token_string is None:
            raise ValueError("API_KEY environment variable not set")

        api_token_string = api_token_string + ':api_token'

        self.headers = {
            'content-type': 'application/json',
            'Authorization' : 'Basic %s' %  b64encode(api_token_string.encode('utf-8')).decode("ascii")
          }

    def get_time_entries(self):
        unix_time_minus_two_hours = int(time.time()) - 7200
        
        response = requests.get(self.TOGGL_API_URL + f'/time_entries?since={unix_time_minus_two_hours}', headers=self.headers)

        # Ensure the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
    
            # TODO: Add a debug mode that prints the details?
            # Or just delete this?
            # Pretty-print the JSON response
            #pretty_json = json.dumps(data, indent=4)
            #print(pretty_json)
        else:
            print(f"Request failed with status code {response.status_code}")

        # Look for non-null stop time to determine whether the time tracker entry actually 
        # has a stop time (if not, it's still in progress).
        if data[0]['stop'] is not None:
            entry_index = 0
        elif len(data) >=2 and data[1]['stop'] is not None:
            entry_index = 1
        else:
            entry_index = -1
            raise DataNotTrustedException("No valid entry found!")
            
        return {"start": data[entry_index]['start'],
                "stop" : data[entry_index]['stop'],
                "description": data[entry_index]['description']}

In [ ]:
x = TogglClient()
x

In [ ]:
gcal = GoogleCalClient()
gcal

In [ ]:
time_entry = None
time_entry = x.get_time_entries()
time_entry

In [ ]:
gcal.create_event(time_entry)